## 1. Install Dependencies

In [ ]:
!pip install -q torch-geometric scikit-learn matplotlib seaborn tqdm

## 2. Configuration

In [ ]:
# ============================================================================
# KAGGLE PATH CONFIGURATION - UPDATE THIS!
# ============================================================================
# Example: If you added dataset named 'graph-iot-dataset', use:
# DATA_DIR = "/kaggle/input/graph-iot-dataset/dataset-processed/ip_gnn"
DATA_DIR = "/kaggle/input/YOUR-DATASET-NAME/dataset-processed/ip_gnn"
OUTPUT_DIR = "/kaggle/working/output/ip_gnn"

# ============================================================================
# PROJECT SETTINGS
# ============================================================================
PROJECT_NAME = "IP-GNN"
SEED = 42
DEVICE = "auto"  # auto, cuda, mps, cpu

# ============================================================================
# DATA SETTINGS
# ============================================================================
SRC_IP_COL = "Src IP"
SRC_PORT_COL = "Src Port"
DST_IP_COL = "Dst IP"
DST_PORT_COL = "Dst Port"
LABEL_COL = "Label"

# ============================================================================
# MODEL SETTINGS
# ============================================================================
HIDDEN_DIM = 128
NUM_CLASSES = 2
NUM_LAYERS = 2
DROPOUT = 0.2
AGGR = "mean"  # mean or sum

# ============================================================================
# GRAPH SETTINGS
# ============================================================================
MAPPING_MODE = "ip_port"  # ip_port or ip_only
ANTI_LEAKAGE_ENABLED = True
ANTI_LEAKAGE_SCOPE = "src_ip_only"  # all_ips or src_ip_only

# ============================================================================
# TRAINING SETTINGS
# ============================================================================
EPOCHS = 50
BATCH_SIZE = 1024
LEARNING_RATE = 0.001
WEIGHT_DECAY = 0.0001
PATIENCE = 10
MIN_DELTA = 0.001

## 3. Imports and Utilities

In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from tqdm import tqdm
import time
import json
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,
    confusion_matrix, roc_curve, auc, classification_report,
    precision_recall_curve, average_precision_score
)
from typing import Dict, Optional, Tuple

print("✅ Imports successful")
print(f"PyTorch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [ ]:
# Utility functions
def set_seed(seed: int = 42):
    """Set random seed for reproducibility."""
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def get_device(device_str: str = "auto") -> torch.device:
    """Get PyTorch device."""
    if device_str == "auto":
        if torch.cuda.is_available():
            device = torch.device("cuda")
            print(f"Using CUDA: {torch.cuda.get_device_name(0)}")
        elif hasattr(torch.backends, 'mps') and torch.backends.mps.is_available():
            device = torch.device("mps")
            print("Using Apple Metal (MPS)")
        else:
            device = torch.device("cpu")
            print("Using CPU")
    else:
        device = torch.device(device_str)
        print(f"Using device: {device}")
    return device

def compute_metrics(y_true: np.ndarray, y_pred: np.ndarray,
                    y_probs: Optional[np.ndarray] = None) -> Dict[str, float]:
    """Compute comprehensive metrics."""
    metrics = {
        "accuracy": accuracy_score(y_true, y_pred),
        "precision": precision_score(y_true, y_pred, pos_label=1, zero_division=0),
        "recall": recall_score(y_true, y_pred, pos_label=1, zero_division=0),
        "f1": f1_score(y_true, y_pred, pos_label=1, zero_division=0)
    }
    
    tn, fp, fn, tp = confusion_matrix(y_true, y_pred, labels=[0, 1]).ravel()
    metrics["true_positive"] = int(tp)
    metrics["true_negative"] = int(tn)
    metrics["false_positive"] = int(fp)
    metrics["false_negative"] = int(fn)
    metrics["far"] = fp / (fp + tn) if (fp + tn) > 0 else 0.0
    metrics["detection_rate"] = metrics["recall"]
    
    if y_probs is not None:
        fpr, tpr, _ = roc_curve(y_true, y_probs)
        metrics["auc"] = auc(fpr, tpr)
        metrics["average_precision"] = average_precision_score(y_true, y_probs)
    
    return metrics

class EarlyStopping:
    """Early stopping callback to prevent overfitting."""
    def __init__(self, patience: int = 10, min_delta: float = 1e-4):
        self.patience = patience
        self.min_delta = min_delta
        self.counter = 0
        self.best_score = None
    
    def __call__(self, score: float) -> bool:
        if self.best_score is None:
            self.best_score = score
            return False
        if score > self.best_score + self.min_delta:
            self.best_score = score
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                return True
        return False

class RandomEdgeSampler:
    """Simple random edge sampler for mini-batch training on full graph."""
    
    def __init__(self, edge_indices: torch.Tensor, batch_size: int, shuffle: bool = True):
        self.edge_indices = edge_indices
        self.batch_size = batch_size
        self.shuffle = shuffle
        
    def __iter__(self):
        indices = self.edge_indices.clone()
        if self.shuffle:
            perm = torch.randperm(len(indices))
            indices = indices[perm]
        
        for i in range(0, len(indices), self.batch_size):
            yield indices[i:i + self.batch_size]
    
    def __len__(self):
        return (len(self.edge_indices) + self.batch_size - 1) // self.batch_size

print("✅ Utilities defined")

## 4. Graph Construction

In [ ]:
def create_endpoint_graph(
    df: pd.DataFrame,
    feature_cols: list,
    src_ip_col: str = SRC_IP_COL,
    dst_ip_col: str = DST_IP_COL,
    label_col: str = LABEL_COL,
    mapping_mode: str = MAPPING_MODE,
    anti_leakage: bool = ANTI_LEAKAGE_ENABLED
) -> Tuple[Data, int]:
    """Create PyG Data object for endpoint-based graph."""
    print(f"\n🔨 Building Endpoint Graph (E-GraphSAGE)...")
    print(f"   Mapping mode: {mapping_mode}")
    print(f"   Anti-leakage: {anti_leakage}")
    
    with tqdm(total=4, desc="📍 Graph construction", ncols=100) as pbar:
        # Step 1: Build endpoint mapping
        endpoint_to_idx, src_indices, dst_indices = _build_endpoint_mapping(
            df, src_ip_col, dst_ip_col, mapping_mode, anti_leakage
        )
        pbar.update(1)
        pbar.set_postfix_str("Built endpoint mapping")
        
        num_nodes = len(endpoint_to_idx)
        num_edges = len(df)
        
        # Step 2: Build edge_index
        edge_index = torch.tensor(
            np.stack([src_indices, dst_indices], axis=0),
            dtype=torch.long
        )
        pbar.update(1)
        pbar.set_postfix_str("Built edge index")
        
        # Step 3: Extract edge features
        edge_attr = df[feature_cols].values.astype(np.float32)
        edge_attr = torch.tensor(edge_attr, dtype=torch.float)
        pbar.update(1)
        pbar.set_postfix_str("Extracted edge features")
        
        # Step 4: Create edge labels
        edge_y = (df[label_col] != 0).astype(int).values
        edge_y = torch.tensor(edge_y, dtype=torch.long)
        pbar.update(1)
        pbar.set_postfix_str("Created edge labels")
    
    # Node features: ones vector (as per E-GraphSAGE)
    num_edge_features = edge_attr.shape[1]
    x = torch.ones((num_nodes, num_edge_features), dtype=torch.float)
    
    # Create PyG Data
    data = Data(
        x=x,
        edge_index=edge_index,
        edge_attr=edge_attr,
        edge_y=edge_y,
        num_nodes=num_nodes
    )
    
    # Statistics
    benign = (edge_y == 0).sum().item()
    attack = (edge_y == 1).sum().item()
    
    print(f"\n✅ Endpoint graph built:")
    print(f"   Nodes (endpoints): {num_nodes:,}")
    print(f"   Edges (flows):     {num_edges:,}")
    print(f"   Edge features:     {num_edge_features}")
    print(f"   Benign edges:      {benign:,} ({benign/num_edges*100:.1f}%)")
    print(f"   Attack edges:      {attack:,} ({attack/num_edges*100:.1f}%)")
    
    return data, num_nodes

def _build_endpoint_mapping(
    df, src_ip_col, dst_ip_col, mapping_mode, anti_leakage
) -> Tuple[Dict, np.ndarray, np.ndarray]:
    """Build endpoint mapping and edge indices."""
    
    # Extract IP columns
    src_ips = df[src_ip_col].astype(str).values
    dst_ips = df[dst_ip_col].astype(str).values
    
    # Apply IP random mapping if enabled (anti-leakage)
    if anti_leakage:
        rng = np.random.RandomState(SEED)
        
        if ANTI_LEAKAGE_SCOPE == "all_ips":
            unique_ips = np.unique(np.concatenate([src_ips, dst_ips]))
        else:  # src_ip_only
            unique_ips = np.unique(src_ips)
        
        ip_map = {ip: f"IP_{i:06d}" for i, ip in enumerate(unique_ips)}
        
        src_ips = np.array([ip_map.get(ip, ip) for ip in src_ips])
        if ANTI_LEAKAGE_SCOPE == "all_ips":
            dst_ips = np.array([ip_map.get(ip, ip) for ip in dst_ips])
    
    # Build endpoint strings
    if mapping_mode == "ip_port":
        src_ports = df[SRC_PORT_COL].astype(str).values
        dst_ports = df[DST_PORT_COL].astype(str).values
        src_endpoints = [f"{ip}:{port}" for ip, port in zip(src_ips, src_ports)]
        dst_endpoints = [f"{ip}:{port}" for ip, port in zip(dst_ips, dst_ports)]
    else:  # ip_only
        src_endpoints = src_ips.tolist()
        dst_endpoints = dst_ips.tolist()
    
    # Create mapping
    unique_endpoints = sorted(set(src_endpoints + dst_endpoints))
    endpoint_to_idx = {ep: idx for idx, ep in enumerate(unique_endpoints)}
    
    # Convert to indices
    src_indices = np.array([endpoint_to_idx[ep] for ep in src_endpoints], dtype=np.int64)
    dst_indices = np.array([endpoint_to_idx[ep] for ep in dst_endpoints], dtype=np.int64)
    
    return endpoint_to_idx, src_indices, dst_indices

print("✅ Graph construction functions defined")

## 5. Define Model

In [ ]:
class EdgeFeatureSAGEConv(nn.Module):
    """SAGEConv layer that incorporates edge features during aggregation.
    
    Hướng 1: Mỗi layer tự project edge_attr
    - Luôn giữ edge_attr gốc (in_edge_dim)
    - Thêm lin_edge: Linear(in_edge_dim -> out_dim)
    - Aggregate lin_edge(edge_attr) thay vì edge_attr raw
    """
    
    def __init__(self, in_dim: int, out_dim: int, in_edge_dim: int, aggr: str = "mean"):
        super().__init__()
        self.in_dim = in_dim
        self.out_dim = out_dim
        self.in_edge_dim = in_edge_dim
        self.aggr = aggr
        
        self.lin_self = nn.Linear(in_dim, out_dim, bias=False)
        self.lin_neigh = nn.Linear(out_dim, out_dim, bias=False)  # Now takes projected edge dim
        self.lin_edge = nn.Linear(in_edge_dim, out_dim, bias=False)  # Project edge_attr to out_dim
        self.bias = nn.Parameter(torch.zeros(out_dim))
        
    def forward(self, x: torch.Tensor, edge_index: torch.Tensor, 
                edge_attr: torch.Tensor) -> torch.Tensor:
        """Forward with edge features.
        
        Args:
            x: Node features [num_nodes, in_dim]
            edge_index: Edges [2, num_edges]
            edge_attr: Original edge features [num_edges, in_edge_dim] (always raw)
            
        Returns:
            Updated node features [num_nodes, out_dim]
        """
        src, dst = edge_index
        num_nodes = x.size(0)
        
        # Self transformation
        out_self = self.lin_self(x)
        
        # Project edge features: edge_attr (in_edge_dim) -> projected (out_dim)
        edge_projected = self.lin_edge(edge_attr)
        
        # Aggregate projected edge features
        aggregated = torch.zeros(num_nodes, self.out_dim, device=x.device)
        
        if self.aggr == "mean":
            ones = torch.ones(edge_index.size(1), device=x.device)
            degree = torch.zeros(num_nodes, device=x.device)
            degree.scatter_add_(0, dst, ones)
            degree = degree.clamp(min=1)
            
            aggregated.scatter_add_(0, dst.unsqueeze(1).expand_as(edge_projected), edge_projected)
            aggregated = aggregated / degree.unsqueeze(1)
        elif self.aggr == "sum":
            aggregated.scatter_add_(0, dst.unsqueeze(1).expand_as(edge_projected), edge_projected)
        
        out_neigh = self.lin_neigh(aggregated)
        out = out_self + out_neigh + self.bias
        
        return out


class EGraphSAGE(nn.Module):
    """E-GraphSAGE for edge classification.
    
    Architecture:
    - K layers of EdgeFeatureSAGEConv
    - Edge representation: concat(z_u, z_v)
    - Edge classifier: Linear(2*hidden_dim -> num_classes)
    """
    
    def __init__(
        self,
        in_dim: int,
        hidden_dim: int = 128,
        num_classes: int = 2,
        num_layers: int = 2,
        dropout: float = 0.2,
        aggr: str = "mean"
    ):
        super().__init__()
        
        self.num_layers = num_layers
        self.hidden_dim = hidden_dim
        self.dropout = dropout
        self.in_edge_dim = in_dim  # Store original edge dimension
        
        # Build layers - all layers receive original edge_attr (in_dim)
        self.convs = nn.ModuleList()
        # First layer: node in_dim -> hidden_dim, edge in_dim -> hidden_dim
        self.convs.append(EdgeFeatureSAGEConv(in_dim, hidden_dim, in_edge_dim=in_dim, aggr=aggr))
        
        # Subsequent layers: node hidden_dim -> hidden_dim, edge still in_dim -> hidden_dim
        for _ in range(num_layers - 1):
            self.convs.append(EdgeFeatureSAGEConv(hidden_dim, hidden_dim, in_edge_dim=in_dim, aggr=aggr))
        
        # Batch normalization
        self.bns = nn.ModuleList()
        for _ in range(num_layers):
            self.bns.append(nn.BatchNorm1d(hidden_dim))
        
        # Edge classifier - output num_classes logits for softmax + cross entropy (paper)
        self.edge_classifier = nn.Sequential(
            nn.Linear(2 * hidden_dim, hidden_dim),
            nn.ReLU(),
            nn.Dropout(dropout),
            nn.Linear(hidden_dim, num_classes)
        )
        
        print(f"   E-GraphSAGE (Hướng 1): edge_dim={in_dim}, {in_dim}→{hidden_dim}x{num_layers}→{num_classes}")
        print(f"   Each layer projects edge_attr: {in_dim}→{hidden_dim}")
        print(f"   Loss: Softmax + CrossEntropy (paper)")
    
    def forward(
        self, 
        x: torch.Tensor, 
        edge_index: torch.Tensor,
        edge_attr: torch.Tensor,
        edge_label_index: torch.Tensor = None
    ) -> torch.Tensor:
        """Forward pass for edge classification.
        
        Args:
            x: Node features [num_nodes, in_dim]
            edge_index: All edges [2, num_edges]
            edge_attr: Edge features [num_edges, in_edge_dim] - always original!
            edge_label_index: Edges to classify [2, num_target_edges]
            
        Returns:
            Edge logits [num_target_edges, num_classes]
        """
        # Update node embeddings - pass original edge_attr to all layers
        h = x
        for i, conv in enumerate(self.convs):
            # Each layer projects edge_attr internally via lin_edge
            h = conv(h, edge_index, edge_attr)  # edge_attr is always original
            h = self.bns[i](h)
            h = F.relu(h)
            h = F.dropout(h, p=self.dropout, training=self.training)
        
        # Get edges to classify
        if edge_label_index is None:
            edge_label_index = edge_index
        
        src_emb = h[edge_label_index[0]]
        dst_emb = h[edge_label_index[1]]
        
        # Concatenate embeddings
        edge_emb = torch.cat([src_emb, dst_emb], dim=1)
        
        # Classify edges - output [num_edges, num_classes] logits
        edge_logits = self.edge_classifier(edge_emb)
        
        return edge_logits  # [num_edges, num_classes]

print("✅ Model defined")

## 6. Training Functions

In [ ]:
def evaluate_edges_fullbatch(model, data, edge_indices, criterion, device, threshold=0.5):
    """Evaluate model on given edges using full-batch forward pass."""
    model.eval()
    
    with torch.no_grad():
        # Single forward pass for all edges
        logits_all = model(data.x, data.edge_index, data.edge_attr)  # [num_edges, num_classes]
        # Get logits for evaluation edges
        logits = logits_all[edge_indices]
        edge_labels = data.edge_y[edge_indices]
        loss = criterion(logits, edge_labels.long()).item()
        
        # Softmax -> probability of class 1 (attack)
        probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        pred = (probs >= threshold).astype(int)
        true = edge_labels.cpu().numpy()
        
        metrics = compute_metrics(true, pred, y_probs=probs)
        
    return loss, metrics

def evaluate_edges_with_predictions(model, data, edge_indices, criterion, threshold, device):
    """Evaluate and return predictions using full-batch forward pass."""
    model.eval()
    
    with torch.no_grad():
        # Single forward pass for all edges
        logits_all = model(data.x, data.edge_index, data.edge_attr)  # [num_edges, num_classes]
        # Get logits for evaluation edges
        logits = logits_all[edge_indices]
        edge_labels = data.edge_y[edge_indices]
        loss = criterion(logits, edge_labels.long()).item()
        
        # Softmax -> probability of class 1 (attack)
        probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        pred = (probs >= threshold).astype(int)
        true = edge_labels.cpu().numpy()
        
        metrics = compute_metrics(true, pred, y_probs=probs)
        
    return loss, metrics, true, pred, probs

def tune_threshold_edges(model, data, edge_indices, device):
    """Find optimal threshold to maximize F1 score.
    
    With softmax output, we tune threshold on P(attack) = softmax(logits)[:, 1]
    """
    model.eval()
    
    with torch.no_grad():
        # Single forward pass for all edges
        logits_all = model(data.x, data.edge_index, data.edge_attr)  # [num_edges, num_classes]
        # Get logits for validation edges
        logits = logits_all[edge_indices]
        # Softmax -> probability of class 1 (attack)
        probs = torch.softmax(logits, dim=1)[:, 1].cpu().numpy()
        true = data.edge_y[edge_indices].cpu().numpy()
    
    best_t, best_f1 = 0.5, 0.0
    
    thresholds = np.linspace(0.01, 0.99, 99)
    for t in tqdm(thresholds, desc="   Searching threshold", ncols=100, leave=False):
        pred = (probs >= t).astype(int)
        f1 = f1_score(true, pred, zero_division=0)
        
        if f1 > best_f1:
            best_f1 = f1
            best_t = t
    
    return best_t

print("✅ Training functions defined")

## 7. Load Data

In [ ]:
set_seed(SEED)
device = get_device(DEVICE)

print("=" * 70)
print("🔷 IP-BASED GNN PIPELINE (E-GraphSAGE)")
print("=" * 70)

data_dir = Path(DATA_DIR)
print(f"\n📂 [1/5] Loading preprocessed data...")
print(f"   Data directory: {data_dir}")

# Load data with IPs
df = pd.read_csv(data_dir / "data_with_ips.csv")
y = np.load(data_dir / "y.npy")
idx_train = np.load(data_dir / "idx_train.npy")
idx_val = np.load(data_dir / "idx_val.npy")
idx_test = np.load(data_dir / "idx_test.npy")

print(f"   Total samples: {len(df):,}")
print(f"   Train: {len(idx_train):,} | Val: {len(idx_val):,} | Test: {len(idx_test):,}")

## 8. Prepare Features

In [ ]:
print("\n🔧 [2/5] Preparing features...")
ip_cols = [SRC_IP_COL, DST_IP_COL]
feature_cols = [c for c in df.columns if c not in ip_cols and c != LABEL_COL]

# Ensure numeric
for col in feature_cols:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df[feature_cols] = df[feature_cols].replace([np.inf, -np.inf], np.nan).fillna(0.0)

print(f"   Features: {len(feature_cols)}")

## 9. Build Graph

In [ ]:
print("\n🔨 [3/5] Building endpoint graph...")
data, num_nodes = create_endpoint_graph(df, feature_cols)

## 10. Use Pre-Split Indices

In [ ]:
print("\n🎯 [4/5] Using pre-split indices from preprocessing...")

# Use the indices that were already loaded from the preprocessing step
# Convert to tensors
train_idx_t = torch.tensor(idx_train, dtype=torch.long)
val_idx_t = torch.tensor(idx_val, dtype=torch.long)
test_idx_t = torch.tensor(idx_test, dtype=torch.long)

print(f"   Train edges: {len(idx_train):,}")
print(f"   Val edges:   {len(idx_val):,}")
print(f"   Test edges:  {len(idx_test):,}")

## 11. Train Model

In [ ]:
print("\n🚀 [5/5] Training model...")
print("\n" + "=" * 70)
print("🚀 TRAINING IP-BASED GNN (E-GraphSAGE)")
print("=" * 70)

# Move data to device
data = data.to(device)
train_idx_t = train_idx_t.to(device)
val_idx_t = val_idx_t.to(device)
test_idx_t = test_idx_t.to(device)

# Calculate class weights for CrossEntropyLoss
train_edge_y = data.edge_y[train_idx_t]
pos = (train_edge_y == 1).sum().item()
neg = (train_edge_y == 0).sum().item()
total = pos + neg
# Weight inversely proportional to class frequency
class_weights = torch.tensor([total / (2 * neg), total / (2 * pos)], device=device)

print(f"\n📊 Dataset Statistics:")
print(f"   Training edges:   {len(idx_train):,}")
print(f"   Validation edges: {len(idx_val):,}")
print(f"   Test edges:       {len(idx_test):,}")
print(f"   Class distribution: Benign={neg:,} ({neg/(neg+pos)*100:.1f}%), Attack={pos:,} ({pos/(neg+pos)*100:.1f}%)")
print(f"   Class weights:      [{class_weights[0]:.4f}, {class_weights[1]:.4f}]")

# Model
print(f"\n🏗️  Model Configuration:")
print(f"   Hidden dim:  {HIDDEN_DIM}")
print(f"   Num layers:  {NUM_LAYERS}")
print(f"   Dropout:     {DROPOUT}")
print(f"   Aggregation: {AGGR}")

model = EGraphSAGE(
    in_dim=data.x.shape[1],
    hidden_dim=HIDDEN_DIM,
    num_classes=NUM_CLASSES,
    num_layers=NUM_LAYERS,
    dropout=DROPOUT,
    aggr=AGGR
).to(device)

total_params = sum(p.numel() for p in model.parameters())
print(f"   Total params: {total_params:,}")
print(f"   Device:       {device}")

# Optimizer & Loss (Softmax + CrossEntropy as per paper)
optimizer = torch.optim.Adam(
    model.parameters(),
    lr=LEARNING_RATE,
    weight_decay=WEIGHT_DECAY
)
criterion = nn.CrossEntropyLoss(weight=class_weights)

print(f"\n⚙️  Training Configuration:")
print(f"   Epochs:         {EPOCHS}")
print(f"   Learning rate:  {LEARNING_RATE}")
print(f"   Weight decay:   {WEIGHT_DECAY}")
print(f"   Early stopping: {PATIENCE} epochs")
print(f"   Training mode:  Full-batch (1 forward/epoch)")

# Early stopping
early_stopping = EarlyStopping(patience=PATIENCE, min_delta=MIN_DELTA)

# Training history
best_f1 = 0.0
history = {
    'train_loss': [],
    'val_loss': [],
    'val_f1': [],
    'val_accuracy': []
}

print(f"\n🔥 Starting Training (Full-batch mode)...")
print("-" * 70)

epoch_pbar = tqdm(range(1, EPOCHS + 1), desc="Training", unit="epoch", ncols=100)

for epoch in epoch_pbar:
    # Train: 1 forward pass per epoch
    model.train()
    optimizer.zero_grad()
    
    # Single forward pass on entire graph for all edges
    logits_all = model(data.x, data.edge_index, data.edge_attr)  # [num_edges, num_classes]
    
    # Compute loss only on training edges
    loss = criterion(logits_all[train_idx_t], data.edge_y[train_idx_t])
    loss.backward()
    optimizer.step()
    
    train_loss = loss.item()
    
    # Validate: use logits from full forward pass
    val_loss, val_metrics = evaluate_edges_fullbatch(model, data, val_idx_t, criterion, device)
    
    # Update history
    history['train_loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['val_f1'].append(val_metrics['f1'])
    history['val_accuracy'].append(val_metrics['accuracy'])
    
    # Update progress bar
    epoch_pbar.set_postfix({
        'loss': f"{train_loss:.4f}",
        'val_f1': f"{val_metrics['f1']:.4f}",
        'val_acc': f"{val_metrics['accuracy']:.4f}"
    })
    
    # Save best model
    if val_metrics['f1'] > best_f1:
        best_f1 = val_metrics['f1']
        save_path = Path(OUTPUT_DIR) / 'best_model.pt'
        save_path.parent.mkdir(parents=True, exist_ok=True)
        torch.save({
            'epoch': epoch,
            'model_state_dict': model.state_dict(),
            'optimizer_state_dict': optimizer.state_dict(),
            'metrics': val_metrics
        }, save_path)
    
    # Early stopping
    if early_stopping(val_metrics['f1']):
        print(f"\n⚠️  Early stopping at epoch {epoch}")
        break

print(f"\n✅ Training completed! Best validation F1: {best_f1:.4f}")

## 12. Tune Threshold

In [ ]:
print("\n" + "=" * 70)
print("🎯 TUNING DECISION THRESHOLD")
print("=" * 70)
best_threshold = tune_threshold_edges(model, data, val_idx_t, device)
print(f"✅ Optimal threshold: {best_threshold:.4f}")

## 13. Final Evaluation

In [ ]:
print("\n" + "=" * 70)
print("🧪 FINAL EVALUATION ON TEST SET")
print("=" * 70)

start_time = time.time()
test_loss, test_metrics, y_true, y_pred, y_probs = evaluate_edges_with_predictions(
    model, data, test_idx_t, criterion, best_threshold, device
)
inference_time = time.time() - start_time
latency_per_sample = inference_time / len(y_true)

print(f"\n📈 Test Results:")
print(f"   Accuracy:  {test_metrics['accuracy']:.4f}")
print(f"   Precision: {test_metrics['precision']:.4f}")
print(f"   Recall:    {test_metrics['recall']:.4f}")
print(f"   F1 Score:  {test_metrics['f1']:.4f}")
print(f"   AUC:       {test_metrics.get('auc', 0):.4f}")
print(f"   FAR:       {test_metrics['far']:.4f}")
print(f"\n⏱️  Inference Performance:")
print(f"   Total time:   {inference_time:.2f}s")
print(f"   Latency:      {latency_per_sample*1000:.4f} ms/sample")
print(f"   Throughput:   {len(y_true)/inference_time:.2f} samples/sec")

## 14. Save Results

In [ ]:
output_path = Path(OUTPUT_DIR)
output_path.mkdir(parents=True, exist_ok=True)

# Save metrics as JSON
test_metrics['latency_seconds'] = latency_per_sample
test_metrics['latency_ms'] = latency_per_sample * 1000
with open(output_path / 'metrics.json', 'w') as f:
    json.dump(test_metrics, f, indent=2)

# Save classification report
report = classification_report(y_true, y_pred, target_names=['Benign', 'Attack'], digits=4)
with open(output_path / 'classification_report.txt', 'w') as f:
    f.write("=" * 60 + "\n")
    f.write("   IP-GNN CLASSIFICATION REPORT\n")
    f.write("=" * 60 + "\n\n")
    f.write(report)

print(f"\n💾 Results saved to {output_path}/")
print("\n" + "=" * 70)
print("✨ ALL DONE!")
print("=" * 70 + "\n")

## 15. Visualizations

In [ ]:
# Confusion Matrix
fig, ax = plt.subplots(figsize=(8, 6))
cm = confusion_matrix(y_true, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Greens',
            xticklabels=['Benign', 'Attack'],
            yticklabels=['Benign', 'Attack'], ax=ax)
ax.set_title('Confusion Matrix - IP GNN', fontsize=14, fontweight='bold')
ax.set_ylabel('True Label')
ax.set_xlabel('Predicted Label')
plt.tight_layout()
plt.savefig(output_path / 'confusion_matrix.png', dpi=300)
plt.show()

# ROC Curve
fig, ax = plt.subplots(figsize=(8, 6))
fpr, tpr, _ = roc_curve(y_true, y_probs)
roc_auc = auc(fpr, tpr)
ax.plot(fpr, tpr, color='#4CAF50', lw=2.5, label=f'ROC Curve (AUC = {roc_auc:.4f})')
ax.plot([0, 1], [0, 1], color='gray', lw=2, linestyle='--', label='Random')
ax.set_xlabel('False Positive Rate')
ax.set_ylabel('True Positive Rate')
ax.set_title('ROC Curve - IP GNN', fontsize=14, fontweight='bold')
ax.legend()
ax.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(output_path / 'roc_curve.png', dpi=300)
plt.show()

# Training History
if len(history['train_loss']) > 0:
    fig, axes = plt.subplots(1, 3, figsize=(18, 5))
    epochs = range(1, len(history['train_loss']) + 1)
    
    axes[0].plot(epochs, history['train_loss'], 'b-', label='Train Loss', linewidth=2)
    axes[0].plot(epochs, history['val_loss'], 'r-', label='Val Loss', linewidth=2)
    axes[0].set_xlabel('Epoch')
    axes[0].set_ylabel('Loss')
    axes[0].set_title('Training & Validation Loss')
    axes[0].legend()
    axes[0].grid(True, alpha=0.3)
    
    axes[1].plot(epochs, history['val_f1'], 'g-', label='Val F1', linewidth=2)
    axes[1].set_xlabel('Epoch')
    axes[1].set_ylabel('F1 Score')
    axes[1].set_title('Validation F1 Score')
    axes[1].legend()
    axes[1].grid(True, alpha=0.3)
    
    axes[2].plot(epochs, history['val_accuracy'], 'm-', label='Val Accuracy', linewidth=2)
    axes[2].set_xlabel('Epoch')
    axes[2].set_ylabel('Accuracy')
    axes[2].set_title('Validation Accuracy')
    axes[2].legend()
    axes[2].grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.savefig(output_path / 'training_history.png', dpi=300)
    plt.show()

print("✅ Visualizations complete!")